In [ ]:
!pip install rdkit-pypi
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from rdkit import DataStructs
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from scipy import stats
from scipy.stats import zscore
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from numpy import linalg as LA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 70.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Suzuki_Project/SMC_Dataset.csv')
df.head()

# Do not include catalyst because catalyst is the same for each
df = df[['Reactant_1_Short_Hand','Reactant_2_Name','Ligand_Short_Hand',\
         'Reagent_1_Short_Hand','Solvent_1_Short_Hand','Product_Yield_PCT_Area_UV']]
# Convert objects to strings
#df = df.applymap(str)

# Remove any rows that contain "None"
#columns = df.columns
#for i in df.columns:
#  df = df[df[i].str.contains("None") == False]

# Convert 'Product_Yield_PCT_Area_UV'
#df['Product_Yield_PCT_Area_UV'] = df['Product_Yield_PCT_Area_UV'].astype(float)

# For now, choose 30% as success threshold so that the dataset is evenish
#df['Product_Yield_PCT_Area_UV'][df['Product_Yield_PCT_Area_UV'] < 30] = 0
#df['Product_Yield_PCT_Area_UV'][df['Product_Yield_PCT_Area_UV'] >= 30] = 1

In [ ]:
df.head(3)

,Reactant_1_Short_Hand,Reactant_2_Name,Ligand_Short_Hand,Reagent_1_Short_Hand,Solvent_1_Short_Hand,Product_Yield_PCT_Area_UV
0,"1a, 6-Cl-Q","2a, Boronic Acid",P(tBu)3,NaOH,MeCN,4.76
1,"1a, 6-Cl-Q","2a, Boronic Acid",P(Ph)3,NaOH,MeCN,4.12
2,"1a, 6-Cl-Q","2a, Boronic Acid",AmPhos,NaOH,MeCN,2.58


In [ ]:
temp = pd.read_csv('/content/drive/MyDrive/Suzuki_Project/SMC_Dataset.csv')
temp.head()

,Reaction_No,Reactant_1_Name,Reactant_1_Short_Hand,Reactant_1_eq,Reactant_1_mmol,Reactant_2_Name,Reactant_2_eq,Catalyst_1_Short_Hand,Catalyst_1_eq,Ligand_Short_Hand,Ligand_eq,Reagent_1_Short_Hand,Reagent_1_eq,Solvent_1_Short_Hand,Product_Yield_PCT_Area_UV,Product_Yield_Mass_Ion_Count,Unnamed: 16,Unnamed: 17
0,1,6-chloroquinoline,"1a, 6-Cl-Q",1.0,0.0004,"2a, Boronic Acid",1.0,Pd(OAc)2,0.0625,P(tBu)3,0.125,NaOH,2.5,MeCN,4.76,6262.06,NaN,NaN
1,2,6-chloroquinoline,"1a, 6-Cl-Q",1.0,0.0004,"2a, Boronic Acid",1.0,Pd(OAc)2,0.0625,P(Ph)3,0.125,NaOH,2.5,MeCN,4.12,13245.57,NaN,NaN
2,3,6-chloroquinoline,"1a, 6-Cl-Q",1.0,0.0004,"2a, Boronic Acid",1.0,Pd(OAc)2,0.0625,AmPhos,0.125,NaOH,2.5,MeCN,2.58,3009.17,NaN,NaN
3,4,6-chloroquinoline,"1a, 6-Cl-Q",1.0,0.0004,"2a, Boronic Acid",1.0,Pd(OAc)2,0.0625,P(Cy)3,0.125,NaOH,2.5,MeCN,4.44,30860.70,NaN,NaN
4,5,6-chloroquinoline,"1a, 6-Cl-Q",1.0,0.0004,"2a, Boronic Acid",1.0,Pd(OAc)2,0.0625,P(o-Tol)3,0.125,NaOH,2.5,MeCN,1.95,2486.31,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reactant_1_Short_Hand      5760 non-null   object 
 1   Reactant_2_Name            5760 non-null   object 
 2   Ligand_Short_Hand          5760 non-null   object 
 3   Reagent_1_Short_Hand       5760 non-null   object 
 4   Solvent_1_Short_Hand       5760 non-null   object 
 5   Product_Yield_PCT_Area_UV  5760 non-null   float64
dtypes: float64(1), object(5)
memory usage: 270.1+ KB


In [ ]:
df.describe()

,Product_Yield_PCT_Area_UV
count,5760.000000
mean,40.109446
std,28.081087
min,0.000000
25%,16.197500
50%,33.580000
75%,62.870000
max,100.000000


In [ ]:
reactant_1 = {'1a, 6-Cl-Q':'C1=CC2=C(C=CC(=C2)Cl)N=C1','1b, 6-Br-Q':'C1=CC2=C(C=CC(=C2)Br)N=C1','1c, 6-OTf-Q':'C1=CC2=C(C=CC(=C2)OS(=O)(=O)C(F)(F)F)N=C1',\
              '1d, 6-I-Q':'C1=CC2=C(C=CC(=C2)I)N=C1','1e, 6-BOH2-Q':'B(C1=CC2=C(C=C1)N=CC=C2)(O)O','1g, 6-BF3K-Q':'[B-](C1=CC2=C(C=C1)N=CC=C2)(F)(F)F.[K+]',\
              '1f, 6-BPin-Q':'B1(OC(C(O1)(C)C)(C)C)C2=CC3=C(C=C2)N=CC=C3'}

reactant_2 = {'2a, Boronic Acid':'B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O','2b, Boronic Ester' : 'B1(OC(C(O1)(C)C)(C)C)C2=C(C=CC3=C2C=NN3C4CCCCO4)C',\
              '2c, Trifluoroborate':'CC1=CC=C(N(C2CCCCO2)N=C3)C3=C1[B-](F)(F)F.[K+]','2d, Bromide':'CC1=C(C2=C(C=C1)N(N=C2)C3CCCCO3)Br'}

ligand = {'P(tBu)3':'CC(C)(C)P(C(C)(C)C)C(C)(C)C','P(Ph)3 ':'C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=CC=CC=C3',\
          'AmPhos':'CC(C)(C)P(C(C)(C)C)C1=CC=C(N(C)C)C=C1','P(Cy)3':'C1CCC(CC1)P(C2CCCCC2)C3CCCCC3',\
          'P(o-Tol)3':'CC1=CC=CC=C1P(C2=CC=CC=C2C)C3=CC=CC=C3C','CataCXium A':'CCCCP(C12CC3CC(C1)CC(C3)C2)C45CC6CC(C4)CC(C6)C5',\
          'SPhos':'COC1=C(C(=CC=C1)OC)C2=CC=CC=C2P(C3CCCCC3)C4CCCCC4','dtbpf':'[Fe].CC(C)(C)P([C]1[CH][CH][CH][CH]1)C(C)(C)C.CC(C)(C)P([C]2[CH][CH][CH][CH]2)C(C)(C)C',\
          'XPhos':'CC(C)C1=CC(=C(C(=C1)C(C)C)C2=CC=CC=C2P(C3CCCCC3)C4CCCCC4)C(C)C','dppf':'[Fe].[CH]1[CH][CH][C]([CH]1)P(c2ccccc2)c3ccccc3.[CH]4[CH][CH][C]([CH]4)P(c5ccccc5)c6ccccc6',\
          'Xantphos':'CC1(C2=C(C(=CC=C2)P(C3=CC=CC=C3)C4=CC=CC=C4)OC5=C1C=CC=C5P(C6=CC=CC=C6)C7=CC=CC=C7)C','None':'0'}

reagent = {'NaOH':'[OH-].[Na+]','NaHCO3':'C(=O)(O)[O-].[Na+]', 'CsF': '[F-].[Cs+]', 'K3PO4':'[O-]P(=O)([O-])[O-].[K+].[K+].[K+]',\
           'KOH':'[OH-].[K+]', 'LiOtBu':'[Li+].CC(C)(C)[O-]', 'Et3N':'CCN(CC)CC', 'None': '0'}
solvent = {'MeCN':'CC#N','THF':'C1CCOC1', 'DMF':'CN(C)C=O', 'MeOH':'CO', 'MeOH/H2O_V2 9:1':'CO', 'THF_V2':'C1CCOC1'}

In [ ]:
df = df.replace({"Reactant_1_Short_Hand": reactant_1,'Reactant_2_Name':reactant_2,\
                 'Ligand_Short_Hand':ligand,'Reagent_1_Short_Hand':reagent,'Solvent_1_Short_Hand':solvent})

df = df.rename(columns={"Reactant_1_Short_Hand": "Reactant_1", "Reactant_2_Name": "Reactant_2",\
                        "Ligand_Short_Hand": "Ligand","Reagent_1_Short_Hand": "Reagent",\
                        "Solvent_1_Short_Hand": "Solvent",'Product_Yield_PCT_Area_UV':'Percent_Yield'})

In [ ]:
df.head()

,Reactant_1,Reactant_2,Ligand,Reagent,Solvent,Percent_Yield
0,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,CC(C)(C)P(C(C)(C)C)C(C)(C)C,[OH-].[Na+],CC#N,4.76
1,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=CC=CC=C3,[OH-].[Na+],CC#N,4.12
2,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,CC(C)(C)P(C(C)(C)C)C1=CC=C(N(C)C)C=C1,[OH-].[Na+],CC#N,2.58
3,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,C1CCC(CC1)P(C2CCCCC2)C3CCCCC3,[OH-].[Na+],CC#N,4.44
4,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,CC1=CC=CC=C1P(C2=CC=CC=C2C)C3=CC=CC=C3C,[OH-].[Na+],CC#N,1.95


In [ ]:
df.to_csv(r'/content/drive/MyDrive/Suzuki_Project/Suzuki_regression.csv')

In [ ]:
df['Percent_Yield'][df['Percent_Yield'] < 30] = 0
df['Percent_Yield'][df['Percent_Yield'] >= 30] = 1

df = df.rename(columns={'Percent_Yield':'Success'})

<ipython-input-10-d8257d75ef2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Percent_Yield'][df['Percent_Yield'] < 30] = 0
<ipython-input-10-d8257d75ef2d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Percent_Yield'][df['Percent_Yield'] >= 30] = 1


In [ ]:
df.head(3)

,Reactant_1,Reactant_2,Ligand,Reagent,Solvent,Success
0,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,CC(C)(C)P(C(C)(C)C)C(C)(C)C,[OH-].[Na+],CC#N,0.0
1,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=CC=CC=C3,[OH-].[Na+],CC#N,0.0
2,C1=CC2=C(C=CC(=C2)Cl)N=C1,B(C1=C(C=CC2=C1C=NN2C3CCCCO3)C)(O)O,CC(C)(C)P(C(C)(C)C)C1=CC=C(N(C)C)C=C1,[OH-].[Na+],CC#N,0.0


In [ ]:
df.to_csv(r'/content/drive/MyDrive/Suzuki_Project/Suzuki_classification.csv')